# Preproceso de texto

Aspectos a considerar para trabajar con (documentos de) texto:
* Los documentos puede ser de diferentes longitudes, por lo que no podemos tratarlos directamente como vectores de características de dimensión fija.
* Las palabras son variables categóricas con muchos valores posibles (la talla del vocabulario), por lo que sus codificaciones one-hot son de muy alta dimensión y sin noción de similitud natural.
* **Out-of-vocabulary (OOV) words:** podemos encontrar palabras en test que no hayan sido vistas en entrenamiento.

## El modelo bolsa de palabras

El modelo **bag of words** ignora el orden de las palabras en un documento, reduciéndolo a un vector de cuentas (de ocurrencias) de cada **palabra, término o token** en un vocabulario fijo de referencia. Para fijar un vocabulario de referencia de talla reducida, se suelen aplicar varias técnicas de preproceso como: eliminación de signos de puntuación, **lowercasing** (conversión a minúsculas), **stop word removal** (supresión de palabras funcionales) y **word stemming** (lematización).
Llamamos **vector space model** al modelo de representación de documentos que resulta: un espacio vectorial en el que cada documento viene representado mediante un vector de $D$ dimensiones. Formalmente, dado un documento $n$ de longitud $T_n$ y con token $x_{nt}$ en la posición $t$, su representación vectorial, $\tilde{\boldsymbol{x}}_n$, incluye el número de ocurrencias de cada palabra $v$ del vocabulario:
$$\tilde{x}_{nv}=\sum_{t=1}^{T_n}\mathbb{I}(x_{nt}=v)$$
Llamamos **term frequency matrix** a la matriz $D\times N$ donde $\text{TF}_{ij}$ es la frecuencia del término $i$ en el documento $j$.

**Ejemplo:** matriz TF con $D=N=20$; los píxeles más oscuros indican términos más frecuentes
<div align="center"><img src="Figure_1.15.png"/></div>

## Representación TF-IDF

La **representación TF-IDF** es una representación clásica de documentos de texto que mejora la TF (bag of words).

Por un lado, sustituye la TF por la **TF log-escalada** para restar importancia a los términos más frecuentes pues, por lo general, tienen poca carga semántica:
$$\log(\operatorname{TF}_{ij}+1)$$

Por otro lado, introduce la **inverse document frequency (IDF)** para estimar la importancia de un término $i$ en proporción inversa al número de documentos que lo incluyen, $\operatorname{DF}_i$, también log-escalada:
$$\operatorname{IDF}_i=\log\frac{N}{1+\operatorname{DF}_i}$$

La TF-IDF combina la TF log-escalada con la IDF:
$$\operatorname{TFIDF}_{ij}=\log(\operatorname{TF}_{ij}+1)\cdot\operatorname{IDF}_i$$

Aunque la TF-IDF mejora la TF dando más peso a las palabras más informativas, esta representación no tiene en cuenta que una mayor proximidad espacial de las palabras debería indicar mayor similitud semántica, cosa (implícitamente) asumida por la mayoría de modelos predictivos.

## Word embeddings

Sea $\boldsymbol{x}_{nt}\in\{0,1\}^V$ la $t$-ésima palabra del $n$-ésimo documento, representada con un vector disperso one-hot sobre un vocabulario de $V$ palabras. Llamamos **word embedding** de $\boldsymbol{x}_{nt}$ a su transformación en un vector denso de dimensión $K$ reducida, $\boldsymbol{e}_{nt}=\mathbf{E}\,\boldsymbol{x}_{nt}$, donde $\mathbf{E}\in\mathbb{R}^{K\times V}$ es una matriz aprendida para que la proximidad vectorial indique similitud semántica. Por extensión, llamamos **bag of word embeddings** a la suma (o promedio) de los word embeddings de todas sus palabras:
$$\bar{\boldsymbol{e}}_n%
=\sum_{t=1}^{T_n}\boldsymbol{e}_{nt}%
=\mathbf{E}\sum_{t=1}^{T_n}\boldsymbol{x}_{nt}%
=\mathbf{E}\,\tilde{\boldsymbol{x}}_n$$
La bolsa de word embeddings puede integrarse fácilmente en un modelo de regresión logística:
$$p(y=c\mid\boldsymbol{x}; \boldsymbol{\theta})=\mathcal{S}_c(\mathbf{WE}\tilde{\boldsymbol{x}}_n)$$
Además, si se emplea una matriz $\mathbf{E}$ **pre-entrenada,** el modelo es lineal con $\mathbf{W}$ y se simplifica el aprendizaje.

## Palabras fuera de vocabulario

Llamamos **palabras fuera del vocabulario (OOV, out-of-vocabulary)** a palabras encontradas en test que no han sido vistas en entrenamiento; por ejemplo, nombres propios de personas y lugares.

Un heurístico convencional para tratar las palabras OOV consiste en reemplazarlas por un símbolo especial, UNK. Ahora bien, este heurístico renuncia a adivinar parte del significado al menos. Por ejemplo, de amaxofobia (miedo a conducir un vehículo o viajar en él) podríamos adivinar que significa aversión exagerada a alguien o algo gracias al sufijo.

Para adivinar al menos una parte del significado de una palabra desconocida, la alternativa usual al UNK consiste en emplear vocabularios de sub-palabras obtenidos con técnicas de codificación como **byte-pair encoding (BPE).**